In [ ]:
# all import statements needed
import os
import json
import requests
import shutil
import urllib
from requests.auth import HTTPBasicAuth
from dotenv import load_dotenv
import pandas as pd
import numpy as np

In [ ]:
# get planet api key from .env
PLANET_API_KEY = os.getenv('PLANET_API_KEY')

In [ ]:
class Region:
    def __init__(self, name, region, dates, cloud_cover, item_type, search_result = None, images=[], active_request = [], activated = set()):
        self.name = name
        # polygon json that contains coordinates of region of intrest
        self.region = region
        # locations that overlap region of interest
        self.location = {
            "type": "GeometryFilter",
            "field_name": "geometry",
            "config": region
        }
        # json that contains dates of interest
        self.dates = dates
        # json that has cloud cover threshol
        self.cloud_cover = cloud_cover
        # json that combines all previous filters
        self.filters = {
            "type": "AndFilter",
            "config": [self.location, dates, cloud_cover]
        }
        # item-type of images desired
        self.item_type = item_type
        # will contain post of search results
        self.search_result = search_result
        # will contains image id of all images
        self.images = images
        # will contain request for each image
        self.active_request = active_request
        # will contain images that still are activating
        self.activated = activated

In [ ]:
df = pd.read_csv('locations.csv')

In [ ]:
regions = []
for i in df.index:
    regions.append(Region(df['Region Name'][i], json.loads(df['Polygon Coordinates'][i]), json.loads(df['Dates'][i]), json.loads(df['Cloud Coverage'][i]), df['Item Type'][i]))

In [ ]:
# set the item type that we are insterested
for region in regions:
    # create a search with filters that we want for each region
    search = {
    "item_types":[region.item_type],
    "filter":region.filters
    }
    # post the request using our PLANET_API_KEY to authenticate
    search_result = \
        requests.post(
            'https://api.planet.com/data/v1/quick-search',
            auth=HTTPBasicAuth(PLANET_API_KEY, ''),
            json=search
        )
    # add the request to region object
    region.search_result = search_result
    region.images = [feature['id'] for feature in search_result.json()['features']]

In [ ]:
# download all data for all regions

for region in regions:
    ...
    for i, image in enumerate(region.images):
        image_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(region.item_type, image)
        result = \
            requests.get(
                image_url,
                auth=HTTPBasicAuth(PLANET_API_KEY, '')
            )
        links = result.json()[u"analytic"]["_links"]
        self_link = links["_self"]
        activation_link = links["activate"]
        # activate links from
        activate_result = \
            requests.get(
                activation_link,
                auth=HTTPBasicAuth(PLANET_API_KEY, '')
            )
        
        activation_status_result = \
            requests.get(
                self_link,
                auth=HTTPBasicAuth(PLANET_API_KEY, '')
            )

        region.active_requests.append(activation_status_result)
        region.activated.add(i)

In [ ]:
for region in regions:
    while region.activated:
        for i, image in enumerate(region.images):
            if i in region.activated and region.active_requests[i].json()["status"] == 'active':
                url = region.active_requests[i].json()["location"]
                name = region.name+"/images/image" + str(i) +".tif"
                download = requests.get(url, stream=True)
                download.raw.decode_content = True
                with open(name, "wb") as f:
                    shutil.copyfileobj(download.raw, f)
                region.activated.remove(i)